In [0]:
!pip install google-generativeai
!pip install openpyxl

In [0]:
dbutils.library.restartPython()

In [0]:
NAME = '<ENTER_YOUR_NAME>'

In [0]:
import google.generativeai as genai
import json
import pandas as pd
import os
import time
import random

In [0]:
gemini_api_keys = [] 

In [0]:
df = pd.read_csv(f'{NAME}_job_listings.csv')
# remove duplicates from df
df = df.drop_duplicates(subset=['job_posting_url'])
all_jobs = []
output_excel = f'full_jobs_data_{NAME}_with_AI (1).xlsx'
# create new excel or open if already exists
if os.path.exists(output_excel):
       existing_data = pd.read_excel(output_excel)
       all_jobs = existing_data.to_dict(orient='records')
       already_filled = existing_data['job_posting_url'].tolist()
else:
    already_filled = []

prompt = "return [job_title], [company], [location],[job_requirements],[benefits],[company_description],[job_description],[additional_relevant_data],[department],[required_education],[industry]  based on this job description, return each value as string if not available return None. return full result in json format:"
i=0
for index, row in df.iterrows():
    if row['job_posting_url'] not in already_filled:
        time.sleep(random.uniform(5, 10))
        genai.configure(api_key=gemini_api_keys[i % len(gemini_api_keys)])
        model = genai.GenerativeModel(model_name='gemini-1.5-flash')
        # print("here")
        job_description = row['description']
        try:
            response = model.generate_content(prompt + job_description)
            # print(response)
            data = json.loads(response.text[8:-5].replace('\n',''))
            # add '_gemini' to the beginning of the keys to avoid conflicts with the original data
            data = {f'{k}_gemini': v for k, v in data.items()}
            # add the new data to the original data and add to the new excel do not change df
            new_row = {**row, **data}
            all_jobs.append(new_row)
            new_data = pd.DataFrame(all_jobs)
            new_data.to_excel(output_excel, index=False)
            print(f"finished {i}/ {len(df)}")
        except:
            print(f'Error with row {i}')
        i+=1
